## Supervised learning (Classification)

In supervised learning, the task is to infer hidden structure from labeled data, comprised of training examples { $(x_n, y_n)$ }. Classification means the output $y$ takes discrete values.

We demonstrate how to do this in Edward with an example. The script is available [here](https://github.com/blei-lab/edward/blob/master/examples/tf_gp_classification.py).

In [1]:
import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Bernoulli, MultivariateNormalFull, Normal
from edward.util import multivariate_rbf

### Data

Use 25 data points from the crabs data set.

In [2]:
ed.set_seed(42)
# DATA
df = np.loadtxt('data/crabs_train.txt', dtype='float32', delimiter=',')
df[df[:, 0] == -1, 0] = 0  # replace -1 label with 0 label
N = 25  # number of data points
D = df.shape[1] - 1  # number of features
subset = np.random.choice(df.shape[0], N, replace=False)
X_train = df[subset, 1:]
y_train = df[subset, 0]

## Model

Posit the model as Gaussian process classification. For more details on the model, see the [Gaussian process classification tutorial](http://edwardlib.org/tutorials/gp-classification).

In [3]:
def kernel(x):
  mat = []
  for i in range(N):
    mat += [[]]
    xi = x[i, :]
    for j in range(N):
      if j == i:
        mat[i] += [multivariate_rbf(xi, xi)]
      else:
        xj = x[j, :]
        mat[i] += [multivariate_rbf(xi, xj)]

    mat[i] = tf.pack(mat[i])

  return tf.pack(mat)

# MODEL
X = tf.placeholder(tf.float32, [N, D])
f = MultivariateNormalFull(mu=tf.zeros(N), sigma=kernel(X))
y = Bernoulli(logits=f)

## Inference

Perform variational inference. Define the variational model to be a fully factorized normal

In [4]:
# INFERENCE
qf = Normal(mu=tf.Variable(tf.random_normal([N])),
            sigma=tf.nn.softplus(tf.Variable(tf.random_normal([N]))))

We run the inference for 500 iterations.

In [5]:
data = {X: X_train, y: y_train}
inference = ed.KLqp({f: qf}, data)
inference.run(n_iter=500)

Iteration   1 [  0%]: Loss = 96.818
Iteration  50 [ 10%]: Loss = 19.040
Iteration 100 [ 20%]: Loss = 24.368
Iteration 150 [ 30%]: Loss = 19.412
Iteration 200 [ 40%]: Loss = 20.554
Iteration 250 [ 50%]: Loss = 20.292
Iteration 300 [ 60%]: Loss = 24.861
Iteration 350 [ 70%]: Loss = 19.802
Iteration 400 [ 80%]: Loss = 19.789
Iteration 450 [ 90%]: Loss = 23.933
Iteration 500 [100%]: Loss = 22.998


In this case `KLqp` defaults to minimizing the $KL(q\Vert p)$ divergence measure using the reparameterization gradient. For more details on inference, see the $KL(q\Vert p)$ [tutorial](http://edwardlib.org/tutorials/klqp). 
(This example happens to be slow because evaluating and inverting full covariances in Gaussian processes happens to be slow.)